## 可变长参数的坑
- 目标是希望调用第一个方法
- 通常来说，不提倡可变长参数方法的重载，是因为 Java 编译器可能无法决定应该调用哪个目标方法
- 如果我调用 invoke(null, 1)，按理编译器会报错，并且提示这个方法调用有二义性
  - 然而，Java 编译器直接将这个方法调用识别为调用第二个方法？？？

``` Java
void invoke(Object obj, Object... args) { ... }
void invoke(String s, Object obj, Object... args) { ... }

invoke(null, 1);        // 调用第二个 invoke 方法
invoke(null, 1, 2);     // 调用第二个 invoke 方法
invoke(null, new Object[]{1}); // 只有手动绕开可变长参数的语法糖，
                               // 才能调用第一个 invoke 方法
```

---

## 重载与重写
### 重载
- Java 里面不允许同一个类中出现多个名字相同，并且参数类型相同的方法
- 如果需要相同命名，则参数类型必须不同(**重载**)

> 小知识：这个限制可以通过字节码工具绕开。也就是说，在编译完成之后，我们可以再向 class 文件中添加方法名和参数类型相同，而返回类型不同的方法。当这种包括多个方法名相同、参数类型相同，而返回类型不同的方法的类，出现在 Java 编译器的用户类路径上时，它是怎么确定需要调用哪个方法的呢？当前版本的 Java 编译器会直接选取第一个方法名以及参数类型匹配的方法。并且，它会根据所选取方法的返回类型来决定可不可以通过编译，以及需不需要进行值转换等。

#### 重载方法的选取
- 编译过程中即可完成识别
- 分为三个阶段
  - 在不考虑对基本类型自动装拆箱（auto-boxing，auto-unboxing），以及可变长参数的情况下选取重载方法
  - 如果在第 1 个阶段中没有找到适配的方法，那么在允许自动装拆箱，但不允许可变长参数的情况下选取重载方法
  - 如果在第 2 个阶段中没有找到适配的方法，那么在允许自动装拆箱以及可变长参数的情况下选取重载方法
- 如果在一个阶段找到多个适配的方法，会在其中选择一个最为贴切的，而决定贴切程度的一个关键就是**形式参数类型的继承关系**
  - 开头的案例，当传入 null 时，它既可以匹配第一个方法中声明为 Object 的形式参数，也可以匹配第二个方法中声明为 String 的形式参数
  - 由于 String 是 Object 的子类，因此 Java 编译器会认为第二个方法更为贴切

#### 父类子类
- 重载也可以作用于这个类所继承而来的方法
- 如果子类定义了与父类中非私有方法同名的方法，而且这两个方法的参数类型不同，那么在子类中，这两个方法同样构成了重载

### 重写
- 如果子类定义了与父类中非私有方法同名的方法，而且这两个方法的参数类型相同，那么这两个方法之间又是什么关系呢？
  - 如果这两个方法都是静态的，那么子类中的方法隐藏了父类中的方法
  - 如果这两个方法都不是静态的，且都不是私有的，那么子类的方法重写了父类中的方法

---

## JVM 的静态绑定和动态绑定
- Java 虚拟机识别方法的关键在于类名、方法名以及**方法描述符**（method descriptor）
  - 方法描述符：由方法的参数类型以及返回类型所构成
- 同一个类中，如果同时出现多个名字相同且描述符也相同的方法，那么 Java 虚拟机会在类的验证阶段报错
- **Java 虚拟机与 Java 语言不同，它并不限制名字与参数类型相同，但返回类型不同的方法出现在同一个类中**

### 重写
- 如果子类定义了与父类中非私有、非静态方法同名的方法，那么只有当这两个方法的参数类型以及返回类型一致，Java 虚拟机才会判定为重写
- 对于 Java 语言中重写而 Java 虚拟机中非重写的情况，编译器会通过生成**桥接方法**来实现 Java 中的重写语义

### 重载
- 由于对重载方法的区分在编译阶段已经完成，我们可以认为 Java 虚拟机不存在重载这一概念
  - **在 Java 虚拟机语境下并非完全正确**：因为某个类中的重载方法可能被它的子类所重写
    - 在某些文章中，重载也被称为静态绑定（static binding），或者编译时多态（compile-time polymorphism）
    - 重写则被称为动态绑定（dynamic binding）
  - Java 虚拟机中确切地说法
    - 静态绑定指的是在解析时便能够直接识别目标方法的情况
    - 动态绑定则指的是需要在运行过程中根据调用者的动态类型来识别目标方法的情况

### 调用指令
- Java 字节码中与调用相关的指令共有五种
  - invokestatic：用于调用静态方法
  - invokespecial：用于调用私有实例方法、构造器，以及使用 super 关键字调用父类的实例方法或构造器，和所实现接口的默认方法
  - invokevirtual：用于调用非私有实例方法
  - invokeinterface：用于调用接口方法
  - invokedynamic：用于调用动态方法
- 对于 invokestatic 以及 invokespecial 而言，Java 虚拟机能够直接识别具体的目标方法
- 对于 invokevirtual 以及 invokeinterface 而言，在绝大部分情况下，虚拟机需要在执行过程中，根据调用者的动态类型，来确定具体的目标方法
- 唯一的例外在于，如果虚拟机能够确定目标方法有且仅有一个，比如说目标方法被标记为 final，那么它可以不通过动态类型，直接确定目标方法

### 案例
```
interface 客户 {
  boolean isVIP();
}

class 商户 {
  public double 折后价格(double 原价, 客户 某客户) {
    return 原价 * 0.8d;
  }
}

class 奸商 extends 商户 {
  @Override
  public double 折后价格(double 原价, 客户 某客户) {
    if (某客户.isVIP()) {                         // invokeinterface      
      return 原价 * 价格歧视();                    // invokestatic
    } else {
      return super.折后价格(原价, 某客户);          // invokespecial
    }
  }
  public static double 价格歧视() {
    // 咱们的杀熟算法太粗暴了，应该将客户城市作为随机数生成器的种子。
    return new Random()                          // invokespecial
           .nextDouble()                         // invokevirtual
           + 0.8d;
  }
}
```

---

## 调用指令的符号引用
- 在编译过程中，我们并不知道目标方法的具体内存地址
  - Java 编译器会暂时用符号引用来表示该目标方法
  - 这一符号引用包括目标方法所在的类或接口的名字，以及目标方法的方法名和方法描述符
- 符号引用存储在 class 文件的常量池之中
  - 引用分类
    - 接口符号引用
    - 非接口符号引用

```
// 在 奸商.class 的常量池中，#16 为接口符号引用，指向接口方法 "客户.isVIP()"
// 而 #22 为非接口符号引用，指向静态方法 "奸商.价格歧视()"
$ javap -v 奸商.class ...
Constant pool:
...
  #16 = InterfaceMethodref #27.#29        // 客户.isVIP:()Z
...
  #22 = Methodref          #1.#33         // 奸商.价格歧视:()D
...
```
---

### 符号引用替换为实际引用
#### 非接口符号引
- 假定该符号引用所指向的类为 C，则 Java 虚拟机会按照如下步骤进行查找
  1. 在 C 中查找符合名字及描述符的方法
  2. 如果没有找到，在 C 的父类中继续搜索，直至 Object 类
  3. 如果没有找到，在 C 所直接实现或间接实现的接口中搜索，这一步搜索得到的目标方法必须是非私有、非静态的。
     - 并且，如果目标方法在间接实现的接口中，则需满足 C 与该接口之间没有其他符合条件的目标方法
     - 如果有多个符合条件的目标方法，则任意返回其中一个
---

- **静态方法也可以通过子类来调用**
- 子类的静态方法会隐藏（注意与重写区分）父类中的同名、同描述符的静态方法

#### 接口符号引用
- 假定该符号引用所指向的接口为 I，则 Java 虚拟机会按照如下步骤进行查找
  1. 在 I 中查找符合名字及描述符的方法
  2. 如果没有找到，在 Object 类中的公有实例方法中搜索
  3. 如果没有找到，则在 I 的超接口中搜索。这一步的搜索结果的要求与非接口符号引用步骤 3 的要求一致

---

- 对于可以**静态绑定**的方法调用而言，实际引用是一个指向方法的指针
- 对于需要**动态绑定**的方法调用而言，实际引用则是一个方法表的索引(下一节详细介绍方法表)

---

## 实践(桥接方法的例子)
- 通过**javap -v**来查看 class 文件所包含的方法

### 案例1: 重写方法的返回类型不一致
- 运行: javac BridgeDemo.java

```
// Merchant.java
public class Merchant {
    public Number actionPrice(double price) {
        return price * 0.8;
    }
}

// NaiveMerchant.java
public class NaiveMerchant extends Merchant {
    @Override
    public Double actionPrice(double price) {
        return 0.9 * price;
    }

    public static void main(String[] args) {
        Merchant merchant = new NaiveMerchant();
        // price 必须定义成 Number 类型 
        Number price = merchant.actionPrice(40);
        System.out.println(price);
    }
}
```

- 运行: javap -v -c NaiveMerchant

```
Classfile /Users/henry/Downloads/NaiveMerchant.class
  Last modified 2020-4-14; size 667 bytes
  MD5 checksum 13ad4d11404e88f8b15c1f18f236dc06
  Compiled from "NaiveMerchant.java"
public class NaiveMerchant extends Merchant
  minor version: 0
  major version: 52
  flags: ACC_PUBLIC, ACC_SUPER
Constant pool:
   #1 = Methodref          #13.#25        // Merchant."<init>":()V
   #2 = Double             0.9d
   #4 = Methodref          #26.#27        // java/lang/Double.valueOf:(D)Ljava/lang/Double;
   #5 = Class              #28            // NaiveMerchant
   #6 = Methodref          #5.#25         // NaiveMerchant."<init>":()V
   #7 = Double             40.0d
   #9 = Methodref          #13.#29        // Merchant.actionPrice:(D)Ljava/lang/Number;
  #10 = Fieldref           #30.#31        // java/lang/System.out:Ljava/io/PrintStream;
  #11 = Methodref          #32.#33        // java/io/PrintStream.println:(Ljava/lang/Object;)V
  #12 = Methodref          #5.#34         // NaiveMerchant.actionPrice:(D)Ljava/lang/Double;
  #13 = Class              #35            // Merchant
  #14 = Utf8               <init>
  #15 = Utf8               ()V
  #16 = Utf8               Code
  #17 = Utf8               LineNumberTable
  #18 = Utf8               actionPrice
  #19 = Utf8               (D)Ljava/lang/Double;
  #20 = Utf8               main
  #21 = Utf8               ([Ljava/lang/String;)V
  #22 = Utf8               (D)Ljava/lang/Number;
  #23 = Utf8               SourceFile
  #24 = Utf8               NaiveMerchant.java
  #25 = NameAndType        #14:#15        // "<init>":()V
  #26 = Class              #36            // java/lang/Double
  #27 = NameAndType        #37:#19        // valueOf:(D)Ljava/lang/Double;
  #28 = Utf8               NaiveMerchant
  #29 = NameAndType        #18:#22        // actionPrice:(D)Ljava/lang/Number;
  #30 = Class              #38            // java/lang/System
  #31 = NameAndType        #39:#40        // out:Ljava/io/PrintStream;
  #32 = Class              #41            // java/io/PrintStream
  #33 = NameAndType        #42:#43        // println:(Ljava/lang/Object;)V
  #34 = NameAndType        #18:#19        // actionPrice:(D)Ljava/lang/Double;
  #35 = Utf8               Merchant
  #36 = Utf8               java/lang/Double
  #37 = Utf8               valueOf
  #38 = Utf8               java/lang/System
  #39 = Utf8               out
  #40 = Utf8               Ljava/io/PrintStream;
  #41 = Utf8               java/io/PrintStream
  #42 = Utf8               println
  #43 = Utf8               (Ljava/lang/Object;)V

{
  public NaiveMerchant();
    descriptor: ()V
    flags: ACC_PUBLIC
    Code:
      stack=1, locals=1, args_size=1
         0: aload_0
         1: invokespecial #1                  // Method Merchant."<init>":()V
         4: return
      LineNumberTable:
        line 1: 0

  public static void main(java.lang.String[]);
    descriptor: ([Ljava/lang/String;)V
    flags: ACC_PUBLIC, ACC_STATIC
    Code:
      stack=3, locals=3, args_size=1
         0: new           #5                  // class NaiveMerchant
         3: dup
         4: invokespecial #6                  // Method "<init>":()V
         7: astore_1
         8: aload_1
         9: ldc2_w        #7                  // double 40.0d
        12: invokevirtual #9                  // Method Merchant.actionPrice:(D)Ljava/lang/Number;
        15: astore_2
        16: getstatic     #10                 // Field java/lang/System.out:Ljava/io/PrintStream;
        19: aload_2
        20: invokevirtual #11                 // Method java/io/PrintStream.println:(Ljava/lang/Object;)V
        23: return
      LineNumberTable:
        line 8: 0
        line 10: 8
        line 11: 16
        line 12: 23

  // 原生方法
  public java.lang.Double actionPrice(double);
    descriptor: (D)Ljava/lang/Double;
    flags: ACC_PUBLIC
    Code:
      stack=4, locals=3, args_size=2
         0: ldc2_w        #2                  // double 0.9d
         3: dload_1
         4: dmul
         5: invokestatic  #4                  // Method java/lang/Double.valueOf:(D)Ljava/lang/Double;
         8: areturn
      LineNumberTable:
        line 4: 0

  // 桥接方法
  public java.lang.Number actionPrice(double);
    descriptor: (D)Ljava/lang/Number;
    flags: ACC_PUBLIC, ACC_BRIDGE, ACC_SYNTHETIC  // ACC_BRIDGE 桥接标志位
    Code:
      stack=3, locals=3, args_size=2
         0: aload_0
         1: dload_1
         2: invokevirtual #12                     // Method actionPrice:(D)Ljava/lang/Double;
         5: areturn
      LineNumberTable:
        line 1: 0
}
SourceFile: "NaiveMerchant.java"
```

---

### 案例2: 范型参数类型造成的方法参数类型不一致
- 一个类一个文件保存，并使用 javac 编译成 class 文件

```
interface Customer {
    String purchase();
}

class VIP implements Customer {
    @Override
    public String purchase() {
        return "VIP First !";
    }
}

class NOT_VIP implements Customer {
    @Override
    public String purchase() {
        return "VIP First !";
    }
}

abstract class MerchantOther<T extends Customer> {
    public double actionPrice(double price, T customer) {
        return price * 0.08;
    }
}

class VIPOnlyMerchant extends MerchantOther<VIP> {
    @Override
    public double actionPrice(double price, VIP customer) {
        return price * 0.07;
    }
}

public class MethodFind {
    public static void main(String[] args) {
        
    }
}
```

---

- 查看 MerchantOther 类情况
  - javap -v -c MerchantOther

```
Classfile /Users/henry/Downloads/MerchantOther.class
  Last modified 2020-4-14; size 358 bytes
  MD5 checksum 686bc998134b81389c5c2de80e3b770e
  Compiled from "MerchantOther.java"
abstract class MerchantOther<T extends Customer> extends java.lang.Object
  minor version: 0
  major version: 52
  flags: ACC_SUPER, ACC_ABSTRACT
Constant pool:
   #1 = Methodref          #5.#17         // java/lang/Object."<init>":()V
   #2 = Double             0.08d
   #4 = Class              #18            // MerchantOther
   #5 = Class              #19            // java/lang/Object
   #6 = Utf8               <init>
   #7 = Utf8               ()V
   #8 = Utf8               Code
   #9 = Utf8               LineNumberTable
  #10 = Utf8               actionPrice
  #11 = Utf8               (DLCustomer;)D
  #12 = Utf8               Signature
  #13 = Utf8               (DTT;)D
  #14 = Utf8               <T::LCustomer;>Ljava/lang/Object;
  #15 = Utf8               SourceFile
  #16 = Utf8               MerchantOther.java
  #17 = NameAndType        #6:#7          // "<init>":()V
  #18 = Utf8               MerchantOther
  #19 = Utf8               java/lang/Object
{
  MerchantOther();
    descriptor: ()V
    flags:
    Code:
      stack=1, locals=1, args_size=1
         0: aload_0
         1: invokespecial #1                  // Method java/lang/Object."<init>":()V
         4: return
      LineNumberTable:
        line 1: 0

  // T 被换成了 Customer 类型
  public double actionPrice(double, T);
    descriptor: (DLCustomer;)D
    flags: ACC_PUBLIC
    Code:
      stack=4, locals=4, args_size=3
         0: dload_1
         1: ldc2_w        #2                  // double 0.08d
         4: dmul
         5: dreturn
      LineNumberTable:
        line 3: 0
    Signature: #13                          // (DTT;)D
}
Signature: #14                          // <T::LCustomer;>Ljava/lang/Object;
SourceFile: "MerchantOther.java"
```

---

- 查看 VIPOnlyMerchant 类情况
  - javap -v -c VIPOnlyMerchant

```
Classfile /Users/henry/Downloads/VIPOnlyMerchant.class
  Last modified 2020-4-14; size 409 bytes
  MD5 checksum 149c0153e849250f7f36f807ed4d5ed8
  Compiled from "VIPOnlyMerchant.java"
class VIPOnlyMerchant extends MerchantOther<VIP>
  minor version: 0
  major version: 52
  flags: ACC_SUPER
Constant pool:
   #1 = Methodref          #7.#19         // MerchantOther."<init>":()V
   #2 = Double             0.07d
   #4 = Class              #20            // VIP
   #5 = Methodref          #6.#21         // VIPOnlyMerchant.actionPrice:(DLVIP;)D
   #6 = Class              #22            // VIPOnlyMerchant
   #7 = Class              #23            // MerchantOther
   #8 = Utf8               <init>
   #9 = Utf8               ()V
  #10 = Utf8               Code
  #11 = Utf8               LineNumberTable
  #12 = Utf8               actionPrice
  #13 = Utf8               (DLVIP;)D
  #14 = Utf8               (DLCustomer;)D
  #15 = Utf8               Signature
  #16 = Utf8               LMerchantOther<LVIP;>;
  #17 = Utf8               SourceFile
  #18 = Utf8               VIPOnlyMerchant.java
  #19 = NameAndType        #8:#9          // "<init>":()V
  #20 = Utf8               VIP
  #21 = NameAndType        #12:#13        // actionPrice:(DLVIP;)D
  #22 = Utf8               VIPOnlyMerchant
  #23 = Utf8               MerchantOther
{
  VIPOnlyMerchant();
    descriptor: ()V
    flags:
    Code:
      stack=1, locals=1, args_size=1
         0: aload_0
         1: invokespecial #1                  // Method MerchantOther."<init>":()V
         4: return
      LineNumberTable:
        line 1: 0

  // 子类定义的类
  public double actionPrice(double, VIP);
    descriptor: (DLVIP;)D
    flags: ACC_PUBLIC
    Code:
      stack=4, locals=4, args_size=3
         0: dload_1
         1: ldc2_w        #2                  // double 0.07d
         4: dmul
         5: dreturn
      LineNumberTable:
        line 4: 0

  // 桥接方法(父类方法)
  public double actionPrice(double, Customer);
    descriptor: (DLCustomer;)D
    flags: ACC_PUBLIC, ACC_BRIDGE, ACC_SYNTHETIC   // ACC_BRIDGE 桥接标志位
    Code:
      stack=4, locals=4, args_size=3
         0: aload_0
         1: dload_1
         2: aload_3
         3: checkcast     #4                  // class VIP
         6: invokevirtual #5                  // Method actionPrice:(DLVIP;)D
         9: dreturn
      LineNumberTable:
        line 1: 0
}
Signature: #16                          // LMerchantOther<LVIP;>;
SourceFile: "VIPOnlyMerchant.java"
```

---

## 参考资料
- [varargs](https://docs.oracle.com/javase/8/docs/technotes/guides/language/varargs.html)
- [bridgeMethods](https://docs.oracle.com/javase/tutorial/java/generics/bridgeMethods.html)
- [HotSpot/VirtualCalls](https://wiki.openjdk.java.net/display/HotSpot/VirtualCalls)
- [HotSpot/InterfaceCalls](https://wiki.openjdk.java.net/display/HotSpot/InterfaceCalls)

---